In [1]:
statements = [
    "The code snippets are related",
    "The code snippets are part of the same module",
    "The code snippets contain duplicated code",
    "The code snippets contain similar logic",
    "The code snippets are responsible for similar tasks",
    "The code snippets are on the same level of abstraction",
    "There are obvious software architecture problems here",
    "This should probably be refactored",
    "I would want to refactor this",
    "This will be easy to refactor",
    "This could likely lead to bugs in the future",
]

In [2]:
import os
from datetime import datetime
from IPython.display import display
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from study_common import *
from study_notebook_helper import *

study_datasets = os.listdir(STUDY_RESULTS_PATH)
# @interact(dataset=[None] + study_datasets)
@interact(dataset=study_datasets)
def main(name="erik", dataset=study_datasets[0]):
    if len(name) == 0:
        print("Please choose a name / id for the study")
        return
    if dataset is None:
        print("Hey " + name + ", please chose a dataset")
        return
    print(name + " is doing " + str(dataset) + ("\n" * 15))
    study: STUDY_TYPE = None
    with open(STUDY_RESULTS_PATH + dataset, "rb") as data_file:
        study = pickle.load(data_file)
    random.Random(name).shuffle(study[1])  # shuffle study entries seeded by participant name
    r = LocalRepo(study[0])
    print_html("<h1>Study Time!</h1>")
    print("\n" * 5)
    
    current_study_slider = widgets.IntSlider(
        value=0, min=0, max=len(study[1]), step=1,
        description='Entry:',
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        layout={'width': '900px'}
    )
    @interact(current_study_entry_index=current_study_slider)
    def show_entry(current_study_entry_index):
        print_html("""<script>document.getElementsByClassName("jp-Notebook")[0].scrollTop = 350;</script>""")
        if current_study_entry_index >= len(study[1]):
            print_html("<h1>You did it!</h1>")
            return
        study_entry = study[1][current_study_entry_index]
        pattern = study_entry[0]
        m0 = study_entry[1]
        m1 = study_entry[2]
        commits = study_entry[3]
        commit_cells: List[str] = []
        commit_empty_cell = '<td style="border: 1px solid black; background-color: #aaa; width: 50%"></td>'
        for c in commits:
            # 0 message, 1 author, 2 date, 3 hexsha, 4 [0 files, 1 additions, 2 deletions], 5 belongs to method 1, 6 belongs to method 2
            datestring = datetime.utcfromtimestamp(c[2]).strftime('%Y-%m-%d')
            commit_cells.append(f"""<tr>
                {commit_empty_cell if not c[5] else ""}
                <td{' colspan="2"' if c[5] and c[6] else ""} style="border: 1px solid black;"><center>
                    {c[1]}: <a href="https://github.com/{r.name}/commit/{c[3]}" target="_blank">{c[0]}</a><br>
                    <span style="font-family: monospace">
                        {"{:,}".format(c[4][0])} changed files:
                        <span style="color:green">+{"{:,}".format(c[4][1])}</span>
                        <span style="color:red">-{"{:,}".format(c[4][2])}</span>,
                        created on {datestring}
                    </span>
                </center></td>
                {commit_empty_cell if not c[6] else ""}
                </tr>""")

        print_html(f"""{PRISM_HIGHLIGHTING}{OWN_STYLE}
            You will be shown multiple methods from the <a href="{r.url()}" target="_blank" style="color: blue">{study[0]}</a> repository.
            Take a close look at the two presented snippets of code.
            Consider everything, not only the statements, but also variable naming, comments,
            location of this snippet within the project, editing history, and relations with other code.
            Then, please select how closely the presented phrases describe this pair of code pieces.
            <br><br><br><br>
            <table border="1">
              <tr>
                <td>{path_html(r, m0[0])}</td> 
                <td>{path_html(r, m1[0])}</td>
              </tr>
              <tr>
                <td><pre><code class="language-java">{m0[1]}</code></pre></td>
                <td><pre><code class="language-java">{m1[1]}</code></pre></td>
              </tr>
              <tr><td colspan="2"><center>And here are the commits: {len(commits)}</center></td></tr>
              <tr><td colspan="2">
                  <div style="overflow-y: scroll; height:400px;">
                  <table>{"".join(commit_cells)}</table>
                  </div>
              </td></tr>
            </table>
        """)

        print_html("<br><h2>Please chose, how strongly these statements apply to the given pair of methods:</h2><br>")

        print_html(HORIZONTAL_RADIO_BUTTONS)
        print_html("<style>.widget-radio > .widget-label {width: 500px;}</style>")
        vote_options = ['Applies strongly', 'Applies partially', 'Does not apply']
        buttons = {
            str(i):
            # widgets.Box([
            #     widgets.Label(value='Pizza topping with a very long label:'),
                widgets.RadioButtons(options=vote_options, description=statement + ":")
            # ])
            for i, statement in enumerate(statements)
        }
        @interact(**buttons)
        def vote(**kwargs):
            print("Saving...", end="")
            import time
            time.sleep(0.5)
            print("Done! Fake :D")
            for index_str, value in kwargs.items():
                print(index_str, statements[int(index_str)], value, 1 - vote_options.index(value))
            print(kwargs)
        def next(btn):
            current_study_slider.value += 1

        button = widgets.Button(
            description='Click me',
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='forward' # (FontAwesome names without the `fa-` prefix)
        )
        button.on_click(next)
        display(button)
    
print("Done!")

interactive(children=(Text(value='erik', description='name'), Dropdown(description='dataset', options=('junit-…

Done!
